# Perinuclear Mean Intensity - p115 Analysis Pipeline

### Improvements:
- Force all images to grayscale (unless Alyce confirms that they are already)

#### Dependencies

In [1]:
import glob
import os
import re
from collections import defaultdict
from dataclasses import dataclass
from pathlib import Path
import napari
import numpy as np
import pandas as pd
import tifffile
from tifffile import imread
from scipy.ndimage import binary_fill_holes
from skimage.measure import label, regionprops
from skimage.morphology import dilation, disk, erosion, square
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.segmentation import find_boundaries
from skimage.util import img_as_float

## Latest Pipeline Version

#### PipelineConfig Dataclass Definition and Custom Configurations

In [2]:
@dataclass
class PipelineConfig:
    show_viewer: bool = True
    nucleus_plane: int = 3 # Keep between 1 and 6
    erosion_intensity: int = 1
    sigma_est_toggle: bool = True
    denoising_strength: float = 0.8
    denoising_patch_size: int = 11
    denoising_patch_distance: int = 12
    denoising_fast_mode: bool = True
    nucleus_mask_threshold: float = 0.05 # Keep between 0 and 1 (confirm)
    nucleus_area_threshold: int = 500
    desired_donut_width_um: float = 4.5
    desired_outer_donut_width_um: float = 1.5
    overlap_filter: str = 'lenient'  # or 'strict'
    lower_transfection_intensity_threshold: float = 0.05 # Keep between 0 and 1
    upper_transfection_intensity_threshold: float = 0.2 # Keep between 0 and 1
    p115_area_intensity_threshold: float = 0.05 # Keep between 0 and 1 (confirm)

cfg_1 = PipelineConfig(
    lower_transfection_intensity_threshold=0.01,
    upper_transfection_intensity_threshold=0.5
)

cfg_2 = PipelineConfig()

cfg_3 = PipelineConfig()

#### Functions for Data cleaning, Mask creation and Filtering, Analysis

In [3]:
def add_image_safe(viewer, data, **kwargs):
    if viewer:
        return viewer.add_image(data, **kwargs)

def add_labels_safe(viewer, data, **kwargs):
    if viewer:
        return viewer.add_labels(data, **kwargs)

In [4]:
def load_and_preprocess_images(img_set, cfg: PipelineConfig):
    """
    Loads nuclei, transfection, and p115 images.
    Returns the images and their metadata.
    """
    viewer = napari.Viewer() if cfg.show_viewer else None
    if viewer:
        viewer.theme = 'light'

    tif_files = glob.glob(os.path.join(img_set, '*.tif'))

    def find_channel_file(files, channel, plane):
        return next(
            (f for f in files if f'ch0{channel}' in os.path.basename(f) and f'z{plane}' in os.path.basename(f)),
            None
        )

    images = {
        'nuclei_img': imread(find_channel_file(tif_files, 0, cfg.nucleus_plane)),
        'nuclei_img_path': find_channel_file(tif_files, 0, cfg.nucleus_plane),
        'transfection_img': imread(find_channel_file(tif_files, 1, cfg.nucleus_plane)),
        'p115_img_lower_plane': imread(find_channel_file(tif_files, 2, cfg.nucleus_plane - 1)),
        'p115_img_mid_plane': imread(find_channel_file(tif_files, 2, cfg.nucleus_plane)),
        'p115_img_upper_plane': imread(find_channel_file(tif_files, 2, cfg.nucleus_plane + 1))
    }

    with tifffile.TiffFile(images['nuclei_img_path']) as tif:
        page = tif.pages[0]
        tags = page.tags
        
        x_res = tags['XResolution'].value
        y_res = tags['YResolution'].value
        res_unit = tags['ResolutionUnit'].value
        
        if res_unit == 3:
            x_ppcm = x_res[0] / x_res[1] # Confirm this logic
            y_ppcm = y_res[0] / y_res[1]
            
            x_ppum = x_ppcm / 10000
            y_ppum = y_ppcm / 10000
    
            print('Compare these values to the metadata in Fiji ImageJ:')
            print(f'X pixels/µm: {x_ppum:.4f}')
            print(f'Y pixels/µm: {y_ppum:.4f}')
        else:
            print('Unsupported resolution unit:', res_unit)
    
    microns_per_pixel = (1 / x_ppum + 1 / y_ppum) / 2
    microns_per_pixel_x = 1 / x_ppum
    microns_per_pixel_y = 1 / y_ppum

    metadata = {
        'microns_per_pixel': microns_per_pixel,
        'microns_per_pixel_x': microns_per_pixel_x,
        'microns_per_pixel_y': microns_per_pixel_y
    }
    
    return images, metadata, viewer

In [5]:
def segment_nuclei(images, viewer, cfg: PipelineConfig):
    """
    Applies erosion, denoising, thresholding, and hole filling.
    Segments individual nuclei from the nuclei channel.
    Returns the cleaned, labeled nuclei mask.
    """
    nuclei_img = images['nuclei_img']
    
    add_image_safe(viewer, nuclei_img, name='nuclei')
    eroded = erosion(nuclei_img, disk(cfg.erosion_intensity))
    add_image_safe(viewer, eroded, name='eroded')

    img_float = img_as_float(eroded)
    
    if (cfg.sigma_est_toggle):
        sigma_est = np.mean(estimate_sigma(img_float, channel_axis=None))
    else:
        sigma_est = 1.0
    
    denoised = denoise_nl_means(
        img_float,
        h=cfg.denoising_strength * sigma_est, 
        patch_size=cfg.denoising_patch_size, 
        patch_distance=cfg.denoising_patch_distance,
        fast_mode=cfg.denoising_fast_mode
    )
    
    add_image_safe(viewer, denoised, name='denoised')

    initial_nuclei_mask = denoised > cfg.nucleus_mask_threshold # CONFIRM IF THIS VALUE NEEDS TO ALWAYS BE BETWEEN 0 AND 1. IF SO, ADD A NOTE IN THE PIPELINECONFIG CLASS.
    add_labels_safe(viewer, initial_nuclei_mask.astype(int), name='initial_nuclei_mask')

    filled_mask = binary_fill_holes(initial_nuclei_mask)
    filled_mask = filled_mask.astype(np.uint8)
    add_labels_safe(viewer, filled_mask, name='hole_filled_mask')

    labeled_mask = label(filled_mask)
    add_labels_safe(viewer, labeled_mask, name='labeled_nuclei_mask')

    image_height, image_width = labeled_mask.shape
    cleaned_nuclei_mask = np.zeros_like(labeled_mask)
    
    for region in regionprops(labeled_mask):
        min_row, min_column, max_row, max_column = region.bbox
        touches_edge = (min_row == 0 or min_column == 0 or max_row == image_height or max_column == image_width)
        
        if not touches_edge and region.area >= cfg.nucleus_area_threshold:
            cleaned_nuclei_mask[labeled_mask == region.label] = region.label 
            
    add_labels_safe(viewer, cleaned_nuclei_mask, name='cleaned_nuclei_mask')
    
    return cleaned_nuclei_mask

In [6]:
def create_donut_masks(cleaned_nuclei_mask, metadata, viewer, cfg: PipelineConfig):
    """
    Creates inner and outer donut masks around each nucleus.
    Ensures that inner and outer donuts share the same labels as their nuclei.
    """
    microns_per_pixel = metadata['microns_per_pixel']
    inner_n_pixels = int(cfg.desired_donut_width_um / microns_per_pixel)
    outer_n_pixels = int(cfg.desired_outer_donut_width_um / microns_per_pixel)
    
    inner_donut_mask = np.zeros_like(cleaned_nuclei_mask, dtype=np.uint16)
    outer_donut_mask = np.zeros_like(cleaned_nuclei_mask, dtype=np.uint16)

    for region in regionprops(cleaned_nuclei_mask):
        nucleus = (cleaned_nuclei_mask == region.label)
        
        dilated_inner = dilation(nucleus, disk(inner_n_pixels))
        inner_donut = dilated_inner & ~nucleus
        inner_donut_mask[inner_donut] = region.label
        
        excluded_area = nucleus | inner_donut
        dilated_outer = dilation(excluded_area, disk(outer_n_pixels))
        outer_donut = dilated_outer & ~excluded_area
        outer_donut_mask[outer_donut] = region.label

    add_labels_safe(viewer, inner_donut_mask, name='inner_donuts')
    add_labels_safe(viewer, outer_donut_mask, name='outer_donuts')

    return inner_donut_mask, outer_donut_mask

In [7]:
def apply_overlap_filter(cleaned_nuclei_mask, inner_donut_mask, outer_donut_mask, viewer, cfg: PipelineConfig):
    """
    Filters out overlapping regions based on the specified method.
    
    Overlap filter modes:
      - "strict": Removes any cell regions that overlap.
      - "lenient": Removes only cell regions whose outer donut overlaps another's inner donut (and vice versa).
      
    Returns the filtered masks.
    """
    regions = regionprops(cleaned_nuclei_mask) # CONFIRM ALL THE CODE BELOW WITH THE PROPOSED REVISED VERSION. ENSURE ALL FUNCTIONALITY IS MAINTAINED.
    
    def remove_labels(mask, labels_to_remove):
        output = np.copy(mask)
        output[np.isin(mask, list(labels_to_remove))] = 0
        return output.astype(np.int32)

    labels_to_remove = set()
    full_regions = []
            
    if cfg.overlap_filter == 'strict':
        for region in regions:
            combined_mask = (
                (cleaned_nuclei_mask == region.label) |
                (inner_donut_mask == region.label) |
                (outer_donut_mask == region.label)
            )
            full_regions.append({
                'label': region.label,
                'mask': combined_mask
            })
            
        for i, region_a in enumerate(full_regions):
            dilated_a = dilation(region_a['mask'], square(3))
            for j, region_b in enumerate(full_regions):
                if i >= j: # CONFIRM THIS AND THE ONE BELOW
                    continue 
                if np.any(dilated_a & region_b['mask']):
                    labels_to_remove.add(region_a['label'])
                    labels_to_remove.add(region_b['label'])

    elif cfg.overlap_filter == 'lenient':
        for region in regions:
            full_regions.append({
                'label': region.label,
                'nucleus': (cleaned_nuclei_mask == region.label),
                'inner_donut': (inner_donut_mask == region.label),
                'outer_donut': (outer_donut_mask == region.label)
            })
            
        for i, region_a in enumerate(full_regions):
            inner_donut_a = region_a['inner_donut']
            outer_donut_a = region_a['outer_donut']
            for j, region_b in enumerate(full_regions):
                if i == j: # CONFIRM THIS AND THE ONE ABOVE
                    continue
                inner_donut_b = region_b['inner_donut']
                outer_donut_b = region_b['outer_donut']
                if np.any(inner_donut_a & outer_donut_b) or np.any(outer_donut_a & inner_donut_b):
                    labels_to_remove.add(region_a['label'])
                    labels_to_remove.add(region_b['label'])
                    
    else:
        raise ValueError('Error: Please set the overlap_filter attribute to "strict" or "lenient."')

    filtered_nuclei = remove_labels(cleaned_nuclei_mask, labels_to_remove)
    filtered_inner_donuts = remove_labels(inner_donut_mask, labels_to_remove)
    filtered_outer_donuts = remove_labels(outer_donut_mask, labels_to_remove)
    add_labels_safe(viewer, filtered_nuclei, name='filtered_nuclei')
    add_labels_safe(viewer, filtered_inner_donuts, name='filtered_inner_donuts')
    add_labels_safe(viewer, filtered_outer_donuts, name='filtered_outer_donuts')
    
    return filtered_nuclei, filtered_inner_donuts, filtered_outer_donuts

In [8]:
def remove_edge_cells(filtered_nuclei, filtered_inner_donuts, filtered_outer_donuts, viewer):
    """
    Removes cell regions that touch the image border.
    Returns masks that are ready for further filtering via transfection analysis.
    """
    image_height, image_width = filtered_nuclei.shape
    
    pre_tf_nuclei = np.zeros_like(filtered_nuclei)
    pre_tf_inner_donuts = np.zeros_like(filtered_inner_donuts)
    pre_tf_outer_donuts = np.zeros_like(filtered_outer_donuts)

    regions = regionprops(filtered_nuclei)
    for region in regions:
        combined_mask = (
            (filtered_nuclei == region.label) |
            (filtered_inner_donuts == region.label) |
            (filtered_outer_donuts == region.label)
        )
        
        if combined_mask.shape != (image_height, image_width):
            raise ValueError(f'Error: Combined mask for label {region.label} has incorrect shape {combined_mask.shape}. Expected ({image_height}, {image_width}).')
        coords = np.argwhere(combined_mask)
        
        if coords.size == 0:
            continue
        min_row, min_col = coords.min(axis=0) # CONFIRM AXIS=0 AND NP.ARGWHERE
        max_row, max_col = coords.max(axis=0)
        touches_edge = (min_row == 0 or min_col == 0 or max_row == image_height - 1 or max_col == image_width - 1)
        
        if not touches_edge:
            pre_tf_nuclei[filtered_nuclei == region.label] = region.label
            pre_tf_inner_donuts[filtered_inner_donuts == region.label] = region.label
            pre_tf_outer_donuts[filtered_outer_donuts == region.label] = region.label
    add_labels_safe(viewer, pre_tf_nuclei, name='pre_transfection_nuclei')
    add_labels_safe(viewer, pre_tf_inner_donuts, name='pre_transfection_inner_donuts')
    add_labels_safe(viewer, pre_tf_outer_donuts, name='pre_transfection_outer_donuts')

    return pre_tf_nuclei, pre_tf_inner_donuts, pre_tf_outer_donuts

In [9]:
def filter_by_transfection(pre_tf_nuclei, pre_tf_inner_donuts, pre_tf_outer_donuts, images, viewer, cfg: PipelineConfig):
    """
    Filters out cells with transfection levels outside the specified thresholds.
    Returns masks for nuclei, inner donuts, and outer donuts after filtering.
    """
    transfection_img = images['transfection_img']
    
    add_image_safe(
        viewer,
        transfection_img,
        name='transfection',
        colormap='green',
        contrast_limits=[0, 255],
        blending='additive',     
        opacity=0.7
    )
    transfection_img_norm = (transfection_img - transfection_img.min()) / (transfection_img.max() - transfection_img.min())
    labels = np.unique(pre_tf_nuclei)
    labels = labels[labels != 0]
    final_nuclei = np.zeros_like(pre_tf_nuclei)
    final_inner_donuts = np.zeros_like(pre_tf_inner_donuts)
    final_outer_donuts = np.zeros_like(pre_tf_outer_donuts)
    for label_id in labels:
        nucleus_mask = (pre_tf_nuclei == label_id)
        inner_donut_mask = (pre_tf_inner_donuts == label_id)
        outer_donut_mask = (pre_tf_outer_donuts == label_id)
        mean_intensity = transfection_img_norm[inner_donut_mask].mean()
        if (mean_intensity >= cfg.lower_transfection_intensity_threshold and mean_intensity <= cfg.upper_transfection_intensity_threshold):
            final_nuclei[nucleus_mask] = label_id
            final_inner_donuts[inner_donut_mask] = label_id
            final_outer_donuts[outer_donut_mask] = label_id
    add_labels_safe(viewer, final_nuclei, name='final_nuclei')
    add_labels_safe(viewer, final_inner_donuts, name='final_inner_donuts')
    add_labels_safe(viewer, final_outer_donuts, name='final_outer_donuts')

    return final_nuclei, final_inner_donuts, final_outer_donuts

In [10]:
def analyze_p115_intensity(final_nuclei, final_inner_donuts, final_outer_donuts, images, metadata, viewer, cfg: PipelineConfig):
    """
    Calculates p115 intensity and area statistics for each region.
    Returns a DataFrame with the results.
    """
    p115_img_lower_plane = images['p115_img_lower_plane']
    p115_img_mid_plane = images['p115_img_mid_plane']
    p115_img_upper_plane = images['p115_img_upper_plane']

    p115_imgs = [images['p115_img_lower_plane'], images['p115_img_mid_plane'], images['p115_img_upper_plane']]
    
    add_image_safe(
        viewer,
        p115_img_mid_plane, 
        name='p115',
        colormap='magma',         
        contrast_limits=[0, 255],
        blending='additive',     
        opacity=0.7
    )
    
    microns_per_pixel_x = metadata['microns_per_pixel_x']
    microns_per_pixel_y = metadata['microns_per_pixel_y']
    
    regions_stats = {}
    labels = np.unique(final_nuclei)
    labels = labels[labels != 0]
    
    for label_id in labels:
        nucleus_mask = (final_nuclei == label_id)
        inner_donut_mask = (final_inner_donuts == label_id)
        outer_donut_mask = (final_outer_donuts == label_id)
        regional_mask = nucleus_mask | inner_donut_mask | outer_donut_mask

        for i, p115_img in enumerate(p115_imgs):
            region_info = {}
            plane = cfg.nucleus_plane - 1 + i
            p115_img_norm = (p115_img - p115_img.min()) / (p115_img.max() - p115_img.min())
            
            for mask, mask_name in zip([nucleus_mask, inner_donut_mask, outer_donut_mask, regional_mask],
                                       ['nucleus', 'inner_donut', 'outer_donut', 'regional']):
                p115_overlap_pixels = p115_img_norm[(mask) & (p115_img_norm > 0)]
                region_info[f'{mask_name}_mean_int'] = float(p115_overlap_pixels.mean() if p115_overlap_pixels.size > 0 else 0) # MAKE SURE THEY KNOW THAT THIS MEAN IS ONLY CALCULATED FROM PIXELS WHOSE INTENSITIES ARE GREATER THAN 0
                region_info[f'{mask_name}_max_int'] = float(p115_overlap_pixels.max() if p115_overlap_pixels.size > 0 else 0)
                region_info[f'{mask_name}_area'] = float((p115_overlap_pixels > cfg.p115_area_intensity_threshold).sum() * microns_per_pixel_x * microns_per_pixel_y)
                
            regions_stats[f'{label_id}z{plane}'] = region_info

    regions_df = pd.DataFrame(regions_stats).T.reset_index().rename(columns={'index': 'label_id'})

    stat_cols = regions_df.columns.drop('label_id')
    summary_row = {
        'label_id': 'overall_means'
    }
    for col in stat_cols:
        stats_array = np.stack(regions_df[col].values) # The logic inside this for loop can be tightened. Do so later.
        mean_stats = stats_array.mean(axis=0)
        summary_row[col] = mean_stats.tolist()
    regions_df = pd.concat([regions_df, pd.DataFrame([summary_row])], ignore_index=True)
    
    return regions_df

In [11]:
def run_analysis(img_sets, output_folder_path, cfg: PipelineConfig):
    """The main orchestrator function."""
    all_img_sets_dfs = []

    for img_set in img_sets.iterdir():
        if img_set.is_dir():
            print(f'Analyzing {img_set}')
        
            images, metadata, viewer = load_and_preprocess_images(img_set, cfg)
            cleaned_nuclei_mask = segment_nuclei(images, viewer, cfg)
            inner_donut_mask, outer_donut_mask = create_donut_masks(cleaned_nuclei_mask, metadata, viewer, cfg)
            filtered_nuclei, filtered_inner_donuts, filtered_outer_donuts = apply_overlap_filter(cleaned_nuclei_mask, inner_donut_mask, outer_donut_mask, viewer, cfg)
            pre_tf_nuclei, pre_tf_inner_donuts, pre_tf_outer_donuts = remove_edge_cells(filtered_nuclei, filtered_inner_donuts, filtered_outer_donuts, viewer)
            final_nuclei, final_inner_donuts, final_outer_donuts = filter_by_transfection(pre_tf_nuclei, pre_tf_inner_donuts, pre_tf_outer_donuts, images, viewer, cfg)
            regions_df = analyze_p115_intensity(final_nuclei, final_inner_donuts, final_outer_donuts, images, metadata, viewer, cfg)
        
            all_img_sets_dfs.append(regions_df)

            if viewer:
                for layer in viewer.layers:
                    layer_name = layer.name.lower()
                    if layer_name not in ['nuclei', 'transfection', 'p115'] and 'final' not in layer_name:
                        layer.visible = False

    summary_rows = []
    for i, df in enumerate(all_img_sets_dfs):
        summary_row = df[df['label_id'] == 'overall_means'].copy()
        summary_row['label_id'] = f'ImageSet_{i+1}'
        summary_rows.append(summary_row)
        
    summary_df = pd.concat(summary_rows, ignore_index=True)
    summary_df.rename(columns={'label_id': 'image_set'}, inplace=True)
    
    print('Image set cleaning and analysis completed.')
    for df in all_img_sets_dfs:
        display(df)
    
    output_path = os.path.join(output_folder_path, 'all_image_set_data.xlsx')
    with pd.ExcelWriter(output_path) as writer:
        for i, df in enumerate(all_img_sets_dfs):
            df.to_excel(writer, sheet_name=f'ImageSet_{i+1}', index=False)
        summary_df.to_excel(writer, sheet_name='OverallData', index=False)

    return all_img_sets_dfs, summary_df

#### Paths to Image Sets and Dataset

In [12]:
img_sets = Path('cell-data/22KO_CHC22_GFP')
output_folder_path = 'analyzed-cell-data'

#### Run Full Analysis (change 3rd argument to any of your configurations)

In [13]:
run_analysis(img_sets, output_folder_path, cfg_1)

Analyzing cell-data/22KO_CHC22_GFP/22KO_CHC22_GFP_7
Compare these values to the metadata in Fiji ImageJ:
X pixels/µm: 5.4992
Y pixels/µm: 5.4992


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Analyzing cell-data/22KO_CHC22_GFP/22KO_CHC22_GFP_6
Compare these values to the metadata in Fiji ImageJ:
X pixels/µm: 5.4992
Y pixels/µm: 5.4992
Image set cleaning and analysis completed.


,label_id,nucleus_mean_int,nucleus_max_int,nucleus_area,inner_donut_mean_int,inner_donut_max_int,inner_donut_area,outer_donut_mean_int,outer_donut_max_int,outer_donut_area,regional_mean_int,regional_max_int,regional_area
0,10z2,0.067241,0.662745,19.278496,0.080406,0.819608,30.885274,0.056362,0.270588,6.712752,0.072676,0.819608,56.876522
1,10z3,0.066033,0.588235,12.235066,0.063439,0.388235,17.922718,0.054842,0.360784,3.670520,0.063271,0.588235,33.828304
2,10z4,0.059606,0.333333,6.150601,0.058526,0.301961,8.928291,0.055742,0.458824,2.149403,0.058506,0.458824,17.228295
3,51z2,0.067179,0.447059,24.370929,0.078032,0.596078,35.184081,0.049252,0.203922,3.868926,0.071309,0.596078,63.423936
4,51z3,0.066607,0.439216,17.492837,0.058451,0.317647,15.012757,0.055769,0.207843,2.711555,0.062063,0.439216,35.217149
5,51z4,0.056102,0.266667,8.300004,0.055463,0.239216,8.333072,0.059722,0.211765,1.984065,0.056203,0.266667,18.617141
6,76z2,0.068735,0.788235,25.164554,0.071062,0.694118,27.115551,0.062569,0.282353,8.465343,0.068962,0.788235,60.745449
7,76z3,0.082634,0.803922,29.331090,0.069364,0.862745,22.750609,0.058678,0.282353,4.563349,0.075129,0.862745,56.645048
8,76z4,0.077956,0.803922,18.914750,0.056905,0.266667,9.986459,0.055003,0.243137,2.215539,0.068866,0.803922,31.116749
9,96z2,0.115352,1.000000,87.728729,0.088996,1.000000,57.041861,0.062938,0.345098,11.573711,0.100862,1.000000,156.344301


,label_id,nucleus_mean_int,nucleus_max_int,nucleus_area,inner_donut_mean_int,inner_donut_max_int,inner_donut_area,outer_donut_mean_int,outer_donut_max_int,outer_donut_area,regional_mean_int,regional_max_int,regional_area
0,10z2,0.067241,0.662745,19.278496,0.080406,0.819608,30.885274,0.056362,0.270588,6.712752,0.072676,0.819608,56.876522
1,10z3,0.066033,0.588235,12.235066,0.063439,0.388235,17.922718,0.054842,0.360784,3.670520,0.063271,0.588235,33.828304
2,10z4,0.059606,0.333333,6.150601,0.058526,0.301961,8.928291,0.055742,0.458824,2.149403,0.058506,0.458824,17.228295
3,51z2,0.067179,0.447059,24.370929,0.078032,0.596078,35.184081,0.049252,0.203922,3.868926,0.071309,0.596078,63.423936
4,51z3,0.066607,0.439216,17.492837,0.058451,0.317647,15.012757,0.055769,0.207843,2.711555,0.062063,0.439216,35.217149
5,51z4,0.056102,0.266667,8.300004,0.055463,0.239216,8.333072,0.059722,0.211765,1.984065,0.056203,0.266667,18.617141
6,76z2,0.068735,0.788235,25.164554,0.071062,0.694118,27.115551,0.062569,0.282353,8.465343,0.068962,0.788235,60.745449
7,76z3,0.082634,0.803922,29.331090,0.069364,0.862745,22.750609,0.058678,0.282353,4.563349,0.075129,0.862745,56.645048
8,76z4,0.077956,0.803922,18.914750,0.056905,0.266667,9.986459,0.055003,0.243137,2.215539,0.068866,0.803922,31.116749
9,96z2,0.115352,1.000000,87.728729,0.088996,1.000000,57.041861,0.062938,0.345098,11.573711,0.100862,1.000000,156.344301


([         label_id  nucleus_mean_int  nucleus_max_int  nucleus_area  \
  0            10z2          0.067241         0.662745     19.278496   
  1            10z3          0.066033         0.588235     12.235066   
  2            10z4          0.059606         0.333333      6.150601   
  3            51z2          0.067179         0.447059     24.370929   
  4            51z3          0.066607         0.439216     17.492837   
  5            51z4          0.056102         0.266667      8.300004   
  6            76z2          0.068735         0.788235     25.164554   
  7            76z3          0.082634         0.803922     29.331090   
  8            76z4          0.077956         0.803922     18.914750   
  9            96z2          0.115352         1.000000     87.728729   
  10           96z3          0.092165         0.937255    101.947860   
  11           96z4          0.116297         1.000000    103.502044   
  12  overall_means          0.077992         0.672549     37.86

## 3rd version of pipeline for easier debugging

In [115]:
@dataclass
class PipelineConfig:
    show_viewer: bool = True
    erosion_intensity: int = 1
    sigma_est_toggle: bool = True
    denoising_strength: float = 0.8
    denoising_patch_size: int = 11
    denoising_patch_distance: int = 12
    denoising_fast_mode: bool = True
    nucleus_mask_threshold: float = 0.05 # Keep between 0 and 1 --> CONFIRM THIS
    nucleus_area_threshold: int = 500
    desired_donut_width_um: float = 4.5
    desired_outer_donut_width_um: float = 1.5
    overlap_filter: str = 'lenient'  # or 'strict'
    lower_transfection_intensity_threshold: float = 0.05 # Keep between 0 and 1
    upper_transfection_intensity_threshold: float = 0.2 # Keep between 0 and 1
    p115_area_intensity_threshold: float = 0.05 # Keep between 0 and 1 --> CONFIRM THIS

In [116]:
cfg = PipelineConfig(
    
)

img_set_paths = {
    'Image Set 1': './cell-data/image-set-1',
    'Image Set 2': './cell-data/image-set-2'
}

output_folder_path = 'analyzed-cell-data'

In [113]:
def add_image_safe(viewer, data, **kwargs):
    if viewer:
        return viewer.add_image(data, **kwargs)

def add_labels_safe(viewer, data, **kwargs):
    if viewer:
        return viewer.add_labels(data, **kwargs)

In [98]:
def load_and_preprocess_images(img_set_path, cfg: PipelineConfig):
    """
    Loads nuclei, transfection, and p115 images.
    Returns the images and their metadata.
    """
    # Your image loading and metadata extraction logic goes here.
    # ...
    viewer = napari.Viewer() if cfg.show_viewer else None
    if viewer:
        viewer.theme = 'light'

    tif_files = glob.glob(os.path.join(img_set_path, '*.tif'))
    sorted_tif_files = sorted(tif_files, key=lambda f: re.search(r'ch\d+', f).group())

    nuclei_img = imread(sorted_tif_files[0])
    nuclei_img_path = sorted_tif_files[0]
    transfection_img = imread(sorted_tif_files[1])
    p115_img = imread(sorted_tif_files[2])

    with tifffile.TiffFile(nuclei_img_path) as tif: 
        page = tif.pages[0]
        tags = page.tags
        
        x_res = tags['XResolution'].value
        y_res = tags['YResolution'].value
        res_unit = tags['ResolutionUnit'].value
        
        if res_unit == 3:
            x_ppcm = x_res[0] / x_res[1] # Confirm the mathematical validity of this logic
            y_ppcm = y_res[0] / y_res[1]
            
            x_ppum = x_ppcm / 10000
            y_ppum = y_ppcm / 10000
    
            print('Compare these values to the metadata in Fiji ImageJ:')
            print(f'X pixels/µm: {x_ppum:.4f}')
            print(f'Y pixels/µm: {y_ppum:.4f}')
        else:
            print('Unsupported resolution unit:', res_unit)
    
    microns_per_pixel = (1 / x_ppum + 1 / y_ppum) / 2
    microns_per_pixel_x = 1 / x_ppum
    microns_per_pixel_y = 1 / y_ppum

    metadata = {
        'microns_per_pixel': microns_per_pixel,
        'microns_per_pixel_x': microns_per_pixel_x,
        'microns_per_pixel_y': microns_per_pixel_y
    }
    
    return nuclei_img, transfection_img, p115_img, metadata, viewer

In [121]:
def segment_nuclei(nuclei_img, viewer, cfg: PipelineConfig):
    """
    Applies erosion, denoising, thresholding, and hole filling.
    Segments individual nuclei from the nuclei channel.
    Returns the cleaned, labeled nuclei mask.
    """
    # Your segmentation and cleaning logic (erosion, denoising, etc.) goes here.
    # ...
    add_image_safe(viewer, nuclei_img, name='nuclei')
    eroded = erosion(nuclei_img, disk(cfg.erosion_intensity))
    add_image_safe(viewer, eroded, name='eroded')

    img_float = img_as_float(eroded)
    
    if (cfg.sigma_est_toggle):
        sigma_est = np.mean(estimate_sigma(img_float, channel_axis=None))
    else:
        sigma_est = 1.0
    
    denoised = denoise_nl_means(
        img_float,
        h=cfg.denoising_strength * sigma_est, 
        patch_size=cfg.denoising_patch_size, 
        patch_distance=cfg.denoising_patch_distance,
        fast_mode=cfg.denoising_fast_mode
    )
    
    add_image_safe(viewer, denoised, name='denoised')

    initial_nuclei_mask = denoised > cfg.nucleus_mask_threshold # CONFIRM IF THIS VALUE NEEDS TO ALWAYS BE BETWEEN 0 AND 1. IF SO, ADD A NOTE IN THE PIPELINECONFIG CLASS.
    add_labels_safe(viewer, initial_nuclei_mask.astype(int), name='initial_nuclei_mask')

    filled_mask = binary_fill_holes(initial_nuclei_mask)
    filled_mask = filled_mask.astype(np.uint8)
    add_labels_safe(viewer, filled_mask, name='hole_filled_mask')

    labeled_mask = label(filled_mask)
    add_labels_safe(viewer, labeled_mask, name='labeled_nuclei_mask')

    image_height, image_width = labeled_mask.shape
    cleaned_nuclei_mask = np.zeros_like(labeled_mask)
    
    for region in regionprops(labeled_mask):
        min_row, min_column, max_row, max_column = region.bbox
        touches_edge = (min_row == 0 or min_column == 0 or max_row == image_height or max_column == image_width)
        
        if not touches_edge and region.area >= cfg.nucleus_area_threshold:
            cleaned_nuclei_mask[labeled_mask == region.label] = region.label 
            
    add_labels_safe(viewer, cleaned_nuclei_mask, name='cleaned_nuclei_mask')
    
    return cleaned_nuclei_mask

In [ ]:
def create_donut_masks(cleaned_nuclei_mask, metadata, viewer, cfg: PipelineConfig):
    """
    Creates inner and outer donut masks around each nucleus.
    Ensures that inner and outer donuts share the same labels as their nuclei.
    """
    microns_per_pixel = metadata['microns_per_pixel']
    inner_n_pixels = int(cfg.desired_donut_width_um / microns_per_pixel)
    outer_n_pixels = int(cfg.desired_outer_donut_width_um / microns_per_pixel)
    
    inner_donut_mask = np.zeros_like(cleaned_nuclei_mask, dtype=np.uint16)
    outer_donut_mask = np.zeros_like(cleaned_nuclei_mask, dtype=np.uint16)

    for region in regionprops(cleaned_nuclei_mask):
        nucleus = (cleaned_nuclei_mask == region.label)
        
        dilated_inner = dilation(nucleus, disk(inner_n_pixels))
        inner_donut = dilated_inner & ~nucleus
        inner_donut_mask[inner_donut] = region.label
        
        excluded_area = nucleus | inner_donut
        dilated_outer = dilation(excluded_area, disk(outer_n_pixels))
        outer_donut = dilated_outer & ~excluded_area
        outer_donut_mask[outer_donut] = region.label

    add_labels_safe(viewer, inner_donut_mask, name='inner_donuts')
    add_labels_safe(viewer, outer_donut_mask, name='outer_donuts')

    return inner_donut_mask, outer_donut_mask

In [107]:
def apply_overlap_filter(cleaned_nuclei_mask, inner_donut_mask, outer_donut_mask, viewer, cfg: PipelineConfig):
    """
    Filters out overlapping regions based on the specified method.
    
    Overlap filter modes:
      - "strict": Removes any cell regions that overlap.
      - "lenient": Removes only cell regions whose outer donut overlaps another's inner donut (and vice versa).
      
    Returns the filtered masks.
    """
    regions = regionprops(cleaned_nuclei_mask) # CONFIRM ALL THE CODE BELOW WITH THE PROPOSED REVISED VERSION. ENSURE ALL FUNCTIONALITY IS MAINTAINED.
    
    def remove_labels(mask, labels_to_remove):
        output = np.copy(mask)
        output[np.isin(mask, list(labels_to_remove))] = 0
        return output.astype(np.int32)

    labels_to_remove = set()
    full_regions = []
            
    if cfg.overlap_filter == 'strict':
        for region in regions:
            combined_mask = (
                (cleaned_nuclei_mask == region.label) |
                (inner_donut_mask == region.label) |
                (outer_donut_mask == region.label)
            )
            full_regions.append({
                'label': region.label,
                'mask': combined_mask
            })
            
        for i, region_a in enumerate(full_regions):
            dilated_a = dilation(region_a['mask'], square(3))
            for j, region_b in enumerate(full_regions):
                if i >= j: # CONFIRM THIS AND THE ONE BELOW
                    continue 
                if np.any(dilated_a & region_b['mask']):
                    labels_to_remove.add(region_a['label'])
                    labels_to_remove.add(region_b['label'])

    elif cfg.overlap_filter == 'lenient':
        for region in regions:
            full_regions.append({
                'label': region.label,
                'nucleus': (cleaned_nuclei_mask == region.label),
                'inner_donut': (inner_donut_mask == region.label),
                'outer_donut': (outer_donut_mask == region.label)
            })
            
        for i, region_a in enumerate(full_regions):
            inner_donut_a = region_a['inner_donut']
            outer_donut_a = region_a['outer_donut']
            for j, region_b in enumerate(full_regions):
                if i == j: # CONFIRM THIS AND THE ONE ABOVE
                    continue
                inner_donut_b = region_b['inner_donut']
                outer_donut_b = region_b['outer_donut']
                if np.any(inner_donut_a & outer_donut_b) or np.any(outer_donut_a & inner_donut_b):
                    labels_to_remove.add(region_a['label'])
                    labels_to_remove.add(region_b['label'])
                    
    else:
        raise ValueError('Error: Please set the overlap_filter attribute to "strict" or "lenient."')

    filtered_nuclei = remove_labels(cleaned_nuclei_mask, labels_to_remove)
    filtered_inner_donuts = remove_labels(inner_donut_mask, labels_to_remove)
    filtered_outer_donuts = remove_labels(outer_donut_mask, labels_to_remove)
    add_labels_safe(viewer, filtered_nuclei, name='filtered_nuclei')
    add_labels_safe(viewer, filtered_inner_donuts, name='filtered_inner_donuts')
    add_labels_safe(viewer, filtered_outer_donuts, name='filtered_outer_donuts')
    
    return filtered_nuclei, filtered_inner_donuts, filtered_outer_donuts

In [102]:
def remove_edge_cells(filtered_nuclei, filtered_inner_donuts, filtered_outer_donuts, viewer):
    """
    Removes cell regions that touch the image border.
    Returns masks that are ready for further filtering via transfection analysis.
    """
    image_height, image_width = filtered_nuclei.shape
    
    pre_tf_nuclei = np.zeros_like(filtered_nuclei)
    pre_tf_inner_donuts = np.zeros_like(filtered_inner_donuts)
    pre_tf_outer_donuts = np.zeros_like(filtered_outer_donuts)

    regions = regionprops(filtered_nuclei)
    for region in regions:
        combined_mask = (
            (filtered_nuclei == region.label) |
            (filtered_inner_donuts == region.label) |
            (filtered_outer_donuts == region.label)
        )
        
        if combined_mask.shape != (image_height, image_width):
            raise ValueError(f'Error: Combined mask for label {region.label} has incorrect shape {combined_mask.shape}. Expected ({image_height}, {image_width}).')
        coords = np.argwhere(combined_mask)
        
        if coords.size == 0:
            continue
        min_row, min_col = coords.min(axis=0) # BE SURE TO FULLY UNDERSTAND THIS SYNTAX (AXIS=0 AND NP.ARGWHERE) 
        max_row, max_col = coords.max(axis=0)
        touches_edge = (min_row == 0 or min_col == 0 or max_row == image_height or max_col == image_width)
        
        if not touches_edge:
            pre_tf_nuclei[filtered_nuclei == region.label] = region.label
            pre_tf_inner_donuts[filtered_inner_donuts == region.label] = region.label
            pre_tf_outer_donuts[filtered_outer_donuts == region.label] = region.label
    add_labels_safe(viewer, pre_tf_nuclei, name='pre_transfection_nuclei')
    add_labels_safe(viewer, pre_tf_inner_donuts, name='pre_transfection_inner_donuts')
    add_labels_safe(viewer, pre_tf_outer_donuts, name='pre_transfection_outer_donuts')

    return pre_tf_nuclei, pre_tf_inner_donuts, pre_tf_outer_donuts

In [103]:
def filter_by_transfection(pre_tf_nuclei, pre_tf_inner_donuts, pre_tf_outer_donuts, transfection_img, viewer, cfg: PipelineConfig):
    """
    Filters out cells with transfection levels outside the specified thresholds.
    Returns masks for nuclei, inner donuts, and outer donuts after filtering.
    """
    add_image_safe(
        viewer,
        transfection_img,
        name='transfection',
        colormap='green',
        contrast_limits=[0, 255],
        blending='additive',     
        opacity=0.7
    )
    transfection_img_norm = (transfection_img - transfection_img.min()) / (transfection_img.max() - transfection_img.min())
    labels = np.unique(pre_tf_nuclei)
    labels = labels[labels != 0]
    final_nuclei = np.zeros_like(pre_tf_nuclei)
    final_inner_donuts = np.zeros_like(pre_tf_inner_donuts)
    final_outer_donuts = np.zeros_like(pre_tf_outer_donuts)
    for label_id in labels:
        nucleus_mask = (pre_tf_nuclei == label_id)
        inner_donut_mask = (pre_tf_inner_donuts == label_id)
        outer_donut_mask = (pre_tf_outer_donuts == label_id)
        mean_intensity = transfection_img_norm[inner_donut_mask].mean()
        if (mean_intensity >= cfg.lower_transfection_intensity_threshold and mean_intensity <= cfg.upper_transfection_intensity_threshold):
            final_nuclei[nucleus_mask] = label_id
            final_inner_donuts[inner_donut_mask] = label_id
            final_outer_donuts[outer_donut_mask] = label_id
    add_labels_safe(viewer, final_nuclei, name='final_nuclei')
    add_labels_safe(viewer, final_inner_donuts, name='final_inner_donuts')
    add_labels_safe(viewer, final_outer_donuts, name='final_outer_donuts')

    return final_nuclei, final_inner_donuts, final_outer_donuts

In [104]:
def analyze_p115_intensity(final_nuclei, final_inner_donuts, final_outer_donuts, p115_img, metadata, viewer, cfg: PipelineConfig):
    """
    Calculates p115 intensity and area statistics for each region.
    Returns a DataFrame with the results.
    """
    # Your statistical analysis logic goes here.
    # ...
    add_image_safe(
        viewer,
        p115_img,
        name='p115',
        colormap='magma',         
        contrast_limits=[0, 255],
        blending='additive',     
        opacity=0.7
    )
    
    p115_img_norm = (p115_img - p115_img.min()) / (p115_img.max() - p115_img.min())
    microns_per_pixel_x = metadata['microns_per_pixel_x']
    microns_per_pixel_y = metadata['microns_per_pixel_y']
    regions_stats = {}
    labels = np.unique(final_nuclei)
    labels = labels[labels != 0]
    for label_id in labels:
        nucleus_mask = (final_nuclei == label_id)
        inner_donut_mask = (final_inner_donuts == label_id)
        outer_donut_mask = (final_outer_donuts == label_id)
        regional_mask = nucleus_mask | inner_donut_mask | outer_donut_mask
        region_info = {}

        for mask, mask_name in zip([nucleus_mask, inner_donut_mask, outer_donut_mask, regional_mask],
                                   ['nucleus', 'inner_donut', 'outer_donut', 'regional']):
            p115_overlap_pixels = p115_img_norm[(mask) & (p115_img_norm > 0)]
            region_info[f'{mask_name}_mean_int'] = float(p115_overlap_pixels.mean() if p115_overlap_pixels.size > 0 else 0) # MAKE SURE THEY KNOW THAT THIS MEAN IS ONLY CALCULATED FROM PIXELS WHOSE INTENSITIES ARE GREATER THAN 0
            region_info[f'{mask_name}_max_int'] = float(p115_overlap_pixels.max() if p115_overlap_pixels.size > 0 else 0)
            region_info[f'{mask_name}_area'] = float((p115_overlap_pixels > cfg.p115_area_intensity_threshold).sum() * microns_per_pixel_x * microns_per_pixel_y)
        regions_stats[label_id] = region_info

    regions_df = pd.DataFrame(regions_stats).T.reset_index().rename(columns={'index': 'label_id'})

    stat_cols = regions_df.columns.drop('label_id')
    summary_row = {
        'label_id': 'overall_means'
    }
    for col in stat_cols:
        stats_array = np.stack(regions_df[col].values) # The logic inside this for loop can be tightened. Do so later.
        mean_stats = stats_array.mean(axis=0)
        summary_row[col] = mean_stats.tolist()
    regions_df = pd.concat([regions_df, pd.DataFrame([summary_row])], ignore_index=True)
    
    return regions_df

In [105]:
def run_analysis(img_set_paths, output_folder_path, cfg: PipelineConfig):
    """The main orchestrator function."""
    
    all_img_sets_dfs = []
    
    for img_set, img_set_path in img_set_paths.items():
        print(f'Analyzing {img_set}')
        
        nuclei_img, transfection_img, p115_img, metadata, viewer = load_and_preprocess_images(img_set_path, cfg)

        cleaned_nuclei_mask = segment_nuclei(nuclei_img, viewer, cfg)

        inner_donut_mask, outer_donut_mask = create_donut_masks(cleaned_nuclei_mask, metadata, viewer, cfg)

        filtered_nuclei, filtered_inner_donuts, filtered_outer_donuts = apply_overlap_filter(cleaned_nuclei_mask, inner_donut_mask, outer_donut_mask, viewer, cfg)

        pre_tf_nuclei, pre_tf_inner_donuts, pre_tf_outer_donuts = remove_edge_cells(filtered_nuclei, filtered_inner_donuts, filtered_outer_donuts, viewer)

        final_nuclei, final_inner_donuts, final_outer_donuts = filter_by_transfection(pre_tf_nuclei, pre_tf_inner_donuts, pre_tf_outer_donuts, transfection_img, viewer, cfg)

        regions_df = analyze_p115_intensity(final_nuclei, final_inner_donuts, final_outer_donuts, p115_img, metadata, viewer, cfg)
        
        all_img_sets_dfs.append(regions_df)

        if viewer:
            for layer in viewer.layers:
                layer_name = layer.name.lower()
                if layer_name not in ['nuclei', 'transfection', 'p115'] and 'final' not in layer_name:
                    layer.visible = False

    summary_rows = []
    for i, df in enumerate(all_img_sets_dfs):
        summary_row = df[df['label_id'] == 'overall_means'].copy()
        summary_row['label_id'] = f'ImageSet_{i+1}'
        summary_rows.append(summary_row)
    summary_df = pd.concat(summary_rows, ignore_index=True)
    summary_df.rename(columns={'label_id': 'image_set'}, inplace=True)
    
    print('Image set cleaning and analysis completed.')
    for df in all_img_sets_dfs:
        display(df)
    
    output_path = os.path.join(output_folder_path, 'all_image_set_data.xlsx')
    with pd.ExcelWriter(output_path) as writer:
        for i, df in enumerate(all_img_sets_dfs):
            df.to_excel(writer, sheet_name=f'ImageSet_{i+1}', index=False)
        summary_df.to_excel(writer, sheet_name='OverallData', index=False)

    return all_img_sets_dfs, summary_df

In [122]:
run_analysis(img_set_paths, output_folder_path, cfg)

Analyzing Image Set 1
Compare these values to the metadata in Fiji ImageJ:
X pixels/µm: 5.5440
Y pixels/µm: 5.5440
Analyzing Image Set 2
Compare these values to the metadata in Fiji ImageJ:
X pixels/µm: 5.5440
Y pixels/µm: 5.5440
Image set cleaning and analysis completed.


,label_id,nucleus_mean_int,nucleus_max_int,nucleus_area,inner_donut_mean_int,inner_donut_max_int,inner_donut_area,outer_donut_mean_int,outer_donut_max_int,outer_donut_area,regional_mean_int,regional_max_int,regional_area
0,29,0.162611,1.000000,48.022521,0.144018,1.000000,33.251366,0.072097,0.247059,3.416236,0.150682,1.000000,84.690124
1,31,0.111972,1.000000,58.824335,0.136644,1.000000,47.046454,0.077057,0.333333,6.149225,0.119688,1.000000,112.020014
2,37,0.111970,0.941176,47.567023,0.107938,0.894118,55.505706,0.079188,0.396078,10.411387,0.106701,0.941176,113.484116
3,48,0.109340,0.894118,59.344905,0.142447,1.000000,65.396523,0.094708,0.764706,12.721413,0.122986,1.000000,137.462841
4,overall_means,0.123973,0.958824,53.439696,0.132762,0.973529,50.300012,0.080762,0.435294,8.174565,0.125014,0.985294,111.914274


,label_id,nucleus_mean_int,nucleus_max_int,nucleus_area,inner_donut_mean_int,inner_donut_max_int,inner_donut_area,outer_donut_mean_int,outer_donut_max_int,outer_donut_area,regional_mean_int,regional_max_int,regional_area
0,7,0.136100,0.980392,43.402469,0.189922,1.0,86.219297,0.150033,1.000000,17.764429,0.168649,1.0,147.386194
1,8,0.121567,0.847059,53.000466,0.232463,1.0,70.797430,0.263864,1.000000,18.154856,0.193886,1.0,141.952751
2,10,0.137843,1.000000,44.768963,0.210093,1.0,80.655712,0.091190,0.545098,13.339589,0.173240,1.0,138.764264
3,30,0.100323,1.000000,51.080866,0.211742,1.0,81.469101,0.126703,1.000000,18.219927,0.160605,1.0,150.769895
4,33,0.155165,1.000000,34.422647,0.201104,1.0,61.589860,0.088565,0.588235,11.289848,0.172589,1.0,107.302355
5,45,0.108606,1.000000,46.005315,0.226637,1.0,71.480677,0.122188,1.000000,13.664945,0.170953,1.0,131.150938
6,overall_means,0.126601,0.971242,45.446788,0.211994,1.0,75.368679,0.140424,0.855556,15.405599,0.173320,1.0,136.221066


([        label_id  nucleus_mean_int  nucleus_max_int  nucleus_area  \
  0             29          0.162611         1.000000     48.022521   
  1             31          0.111972         1.000000     58.824335   
  2             37          0.111970         0.941176     47.567023   
  3             48          0.109340         0.894118     59.344905   
  4  overall_means          0.123973         0.958824     53.439696   
  
     inner_donut_mean_int  inner_donut_max_int  inner_donut_area  \
  0              0.144018             1.000000         33.251366   
  1              0.136644             1.000000         47.046454   
  2              0.107938             0.894118         55.505706   
  3              0.142447             1.000000         65.396523   
  4              0.132762             0.973529         50.300012   
  
     outer_donut_mean_int  outer_donut_max_int  outer_donut_area  \
  0              0.072097             0.247059          3.416236   
  1              0.07705

## 2nd version of pipeline for batch analysis

In [3]:
def clean_and_analyze_images(
    img_set_paths,
    output_folder_path,
    show_viewer=False,
    erosion_intensity=1,
    sigma_est_toggle=False, 
    denoising_strength = 0.8,
    denoising_patch_size=11,
    denoising_patch_distance=12, 
    denoising_fast_mode=True, 
    nucleus_mask_threshold=0.05, 
    nucleus_area_threshold=500,
    desired_donut_width_um=4.5, 
    desired_outer_donut_width_um=1.5, 
    strict_overlapping_filter=False,
    lenient_overlapping_filter=True, 
    lower_transfection_intensity_threshold=0.05,
    upper_transfection_intensity_threshold=0.2,
    p115_area_intensity_threshold=0.05
):
    all_img_sets_dfs = []
    for img_set, img_set_path in img_set_paths.items():
        print(f'Analyzing {img_set}')
        tif_files = glob.glob(os.path.join(img_set_path, '*.tif'))
        sorted_tif_files = sorted(tif_files, key=lambda f: re.search(r'ch\d+', f).group())

        nuclei_img = imread(sorted_tif_files[0])
        nuclei_img_path = sorted_tif_files[0]
        transfection_img = imread(sorted_tif_files[1])
        p115_img = imread(sorted_tif_files[2])

        viewer = napari.Viewer() if show_viewer else None
        if viewer:
            viewer.theme = 'light'

        def add_image_safe(data, **kwargs):
            if viewer:
                return viewer.add_image(data, **kwargs)

        def add_labels_safe(data, **kwargs):
            if viewer:
                return viewer.add_labels(data, **kwargs)
        
        add_image_safe(nuclei_img, name='nuclei')

        eroded = erosion(nuclei_img, disk(erosion_intensity))
        add_image_safe(eroded, name='eroded')

        img_float = img_as_float(eroded)
        if (sigma_est_toggle):
            sigma_est = np.mean(estimate_sigma(img_float, channel_axis=None))
        else:
            sigma_est = 1.0
        denoised = denoise_nl_means(
            img_float,
            h=denoising_strength * sigma_est, 
            patch_size=denoising_patch_size, 
            patch_distance=denoising_patch_distance,
            fast_mode=denoising_fast_mode
        )
        add_image_safe(denoised, name='denoised')

        initial_nuclei_mask = denoised > nucleus_mask_threshold
        add_labels_safe(initial_nuclei_mask.astype(int), name='initial_nuclei_mask')

        filled_mask = binary_fill_holes(initial_nuclei_mask)
        filled_mask = filled_mask.astype(np.uint8)
        add_labels_safe(filled_mask, name='hole_filled_mask')

        labeled_mask = label(filled_mask)
        add_labels_safe(labeled_mask, name='labeled_nuclei_mask')

        image_height, image_width = labeled_mask.shape
        cleaned_nuclei_mask = np.zeros_like(labeled_mask)
        for region in regionprops(labeled_mask):
            min_row, min_column, max_row, max_column = region.bbox
            touches_edge = (min_row == 0 or min_column == 0 or max_row == image_height or max_column == image_width)
            if not touches_edge and region.area >= nucleus_area_threshold:
                cleaned_nuclei_mask[labeled_mask == region.label] = region.label 
        add_labels_safe(cleaned_nuclei_mask, name='cleaned_nuclei_mask')

        with tifffile.TiffFile(nuclei_img_path) as tif: 
            page = tif.pages[0]
            tags = page.tags
            x_res = tags['XResolution'].value
            y_res = tags['YResolution'].value
            res_unit = tags['ResolutionUnit'].value
            if res_unit == 3:
                x_ppcm = x_res[0] / x_res[1]
                y_ppcm = y_res[0] / y_res[1]
                x_ppum = x_ppcm / 10000
                y_ppum = y_ppcm / 10000
                print('Compare these values to the metadata in Fiji ImageJ:')
                print(f'X pixels/µm: {x_ppum:.4f}')
                print(f'Y pixels/µm: {y_ppum:.4f}')
            else:
                print('Unsupported resolution unit:', res_unit)
        microns_per_pixel = (1 / x_ppum + 1 / y_ppum) / 2
        n_pixels = int(desired_donut_width_um / microns_per_pixel)
        outer_mask = dilation(cleaned_nuclei_mask, disk(n_pixels))
        donut_mask = outer_mask - cleaned_nuclei_mask
        donut_mask[donut_mask < 0] = 0
        donut_mask = donut_mask.astype(np.uint8)
        add_labels_safe(donut_mask, name='donut')
    
        microns_per_pixel_x = 1 / x_ppum
        microns_per_pixel_y = 1 / y_ppum

        n_pixels = int(desired_outer_donut_width_um / microns_per_pixel)
        outer_donut_mask = np.zeros_like(cleaned_nuclei_mask, dtype=np.uint16)
        regions = regionprops(cleaned_nuclei_mask)
        for region in regions:
            nucleus_mask = (cleaned_nuclei_mask == region.label)
            first_donut_mask = (donut_mask == region.label)
            excluded_area = nucleus_mask | first_donut_mask
            outer_mask = dilation(excluded_area, disk(n_pixels))
            outer_donut = outer_mask & (~excluded_area)
            outer_donut_mask[outer_donut] = region.label
        add_labels_safe(outer_donut_mask, name='outer_donut')

        def remove_labels(mask, labels_to_remove):
            output = np.copy(mask)
            output[np.isin(mask, list(labels_to_remove))] = 0
            return output.astype(np.int32)
            
        if (strict_overlapping_filter):
            full_regions = []
            for region in regions:
                combined_mask = (
                    (cleaned_nuclei_mask == region.label) |
                    (donut_mask == region.label) |
                    (outer_donut_mask == region.label)
                )
                full_regions.append({
                    'label': region.label,
                    'mask': combined_mask
                })
            labels_to_remove = set()
            for i, region_a in enumerate(full_regions):
                dilated_a = dilation(region_a['mask'], square(3))
                for j, region_b in enumerate(full_regions):
                    if i >= j: # CONFIRM THIS AND THE ONE BELOW
                        continue 
                    if np.any(dilated_a & region_b['mask']):
                        labels_to_remove.add(region_a['label'])
                        labels_to_remove.add(region_b['label'])
            cleaned_nuclei_filtered = remove_labels(cleaned_nuclei_mask, labels_to_remove)
            donut_filtered = remove_labels(donut_mask, labels_to_remove)
            outer_donut_filtered = remove_labels(outer_donut_mask, labels_to_remove)
            add_labels_safe(cleaned_nuclei_filtered, name='no_overlap_nuclei')
            add_labels_safe(donut_filtered, name='no_overlap_donuts')
            add_labels_safe(outer_donut_filtered, name='no_overlap_outer_donut')

        if (lenient_overlapping_filter):
            full_regions = []
            for region in regions:
                full_region = {
                    'label': region.label,
                    'nucleus': (cleaned_nuclei_mask == region.label),
                    'donut': (donut_mask == region.label),
                    'outer': (outer_donut_mask == region.label)
                }
                full_regions.append(full_region)
            labels_to_remove = set()
            for i, region_a in enumerate(full_regions):
                inner_donut_a = region_a['donut']
                outer_donut_a = region_a['outer']
                for j, region_b in enumerate(full_regions):
                    if i == j: # CONFIRM THIS AND THE ONE ABOVE
                        continue
                    inner_donut_b = region_b['donut']
                    outer_donut_b = region_b['outer']
                    if np.any(inner_donut_a & outer_donut_b):
                        labels_to_remove.add(region_a['label'])
                        labels_to_remove.add(region_b['label'])
                    elif np.any(outer_donut_a & inner_donut_b):
                        labels_to_remove.add(region_a['label'])
                        labels_to_remove.add(region_b['label'])
            cleaned_nuclei_filtered = remove_labels(cleaned_nuclei_mask, labels_to_remove)
            donut_filtered = remove_labels(donut_mask, labels_to_remove)
            outer_donut_filtered = remove_labels(outer_donut_mask, labels_to_remove)
            add_labels_safe(cleaned_nuclei_filtered, name='filtered_nuclei')
            add_labels_safe(donut_filtered, name='filtered_donuts')
            add_labels_safe(outer_donut_filtered, name='filtered_outer_donuts')

        if not (strict_overlapping_filter or lenient_overlapping_filter):
            raise ValueError('Error: Please set one of the overlap filtering methods to True.')

        image_height, image_width = cleaned_nuclei_filtered.shape
        pre_tf_nuclei = np.zeros_like(cleaned_nuclei_filtered)
        pre_tf_donuts = np.zeros_like(donut_filtered)
        pre_tf_outer_donuts = np.zeros_like(outer_donut_filtered)
        regions = regionprops(cleaned_nuclei_filtered)
        for region in regions: 
            combined = (
                (cleaned_nuclei_filtered == region.label) |
                (donut_filtered == region.label) |
                (outer_donut_filtered == region.label)
            )
            if combined.shape != (image_height, image_width):
                raise ValueError(f'Error: Combined mask for label {region.label} has incorrect shape {combined.shape}. Expected ({image_height}, {image_width}).')
            coords = np.argwhere(combined)
            if coords.size == 0:
                continue
            min_row, min_col = coords.min(axis=0)
            max_row, max_col = coords.max(axis=0)
            touches_edge = (min_row == 0 or min_col == 0 or max_row == image_height or max_col == image_width)
            if not touches_edge:
                pre_tf_nuclei[cleaned_nuclei_filtered == region.label] = region.label
                pre_tf_donuts[donut_filtered == region.label] = region.label
                pre_tf_outer_donuts[outer_donut_filtered == region.label] = region.label
        add_labels_safe(pre_tf_nuclei, name='pre_transfection_nuclei')
        add_labels_safe(pre_tf_donuts, name='pre_transfection_donuts')
        add_labels_safe(pre_tf_outer_donuts, name='pre_transfection_outer_donuts')

        add_image_safe(
            transfection_img,
            name='transfection',
            colormap='green',
            contrast_limits=[0, 255],
            blending='additive',     
            opacity=0.7
        )
        transfection_img_norm = (transfection_img - transfection_img.min()) / (transfection_img.max() - transfection_img.min())
        labels = np.unique(pre_tf_nuclei)
        labels = labels[labels != 0]
        final_nuclei = np.zeros_like(pre_tf_nuclei)
        final_donuts = np.zeros_like(pre_tf_donuts)
        final_outer_donuts = np.zeros_like(pre_tf_outer_donuts)
        for label_id in labels:
            nucleus_mask = (pre_tf_nuclei == label_id)
            donut_mask = (pre_tf_donuts == label_id)
            outer_mask = (pre_tf_outer_donuts == label_id)
            mean_intensity = transfection_img_norm[donut_mask].mean()
            if (mean_intensity >= lower_transfection_intensity_threshold and mean_intensity <= upper_transfection_intensity_threshold):
                final_nuclei[nucleus_mask] = label_id
                final_donuts[donut_mask] = label_id
                final_outer_donuts[outer_mask] = label_id
        add_labels_safe(final_nuclei, name='final_nuclei')
        add_labels_safe(final_donuts, name='final_donuts')
        add_labels_safe(final_outer_donuts, name='final_outer_donuts')

        add_image_safe(
            p115_img,
            name='p115',
            colormap='magma',         
            contrast_limits=[0, 255],
            blending='additive',     
            opacity=0.7
        )
        p115_img_norm = (p115_img - p115_img.min()) / (p115_img.max() - p115_img.min())
        regions_stats = {}
        labels = np.unique(final_nuclei)
        labels = labels[labels != 0]
        for label_id in labels:
            nucleus_mask = (final_nuclei == label_id)
            donut_mask = (final_donuts == label_id)
            outer_donut_mask = (final_outer_donuts == label_id)
            regional_mask = nucleus_mask | donut_mask | outer_donut_mask
            region_info = {}

            for mask, mask_name in zip([nucleus_mask, donut_mask, outer_donut_mask, regional_mask],
                                       ['nucleus', 'inner_donut', 'outer_donut', 'regional']):
                p115_overlap_pixels = p115_img_norm[(mask) & (p115_img_norm > 0)]
                region_info[f'{mask_name}_mean_int'] = float(p115_overlap_pixels.mean() if p115_overlap_pixels.size > 0 else 0) # MAKE SURE THEY KNOW THAT THIS MEAN IS ONLY CALCULATED FROM PIXELS WHOSE INTENSITIES ARE GREATER THAN 0
                region_info[f'{mask_name}_max_int'] = float(p115_overlap_pixels.max() if p115_overlap_pixels.size > 0 else 0)
                region_info[f'{mask_name}_area'] = float((p115_overlap_pixels > p115_area_intensity_threshold).sum() * microns_per_pixel_x * microns_per_pixel_y)
            regions_stats[label_id] = region_info

        regions_df = pd.DataFrame(regions_stats).T.reset_index().rename(columns={'index': 'label_id'})

        stat_cols = regions_df.columns.drop('label_id')
        summary_row = {
            'label_id': 'overall_means'
        }
        for col in stat_cols:
            stats_array = np.stack(regions_df[col].values)
            mean_stats = stats_array.mean(axis=0)
            summary_row[col] = mean_stats.tolist()
        regions_df = pd.concat([regions_df, pd.DataFrame([summary_row])], ignore_index=True)
        all_img_sets_dfs.append(regions_df)

        if viewer:
            for layer in viewer.layers:
                layer_name = layer.name.lower()
                if layer_name not in ['nuclei', 'transfection', 'p115'] and 'final' not in layer_name:
                    layer.visible = False

    summary_rows = []
    for i, df in enumerate(all_img_sets_dfs):
        summary_row = df[df['label_id'] == 'overall_means'].copy()
        summary_row['label_id'] = f'ImageSet_{i+1}'
        summary_rows.append(summary_row)
    summary_df = pd.concat(summary_rows, ignore_index=True)
    summary_df.rename(columns={'label_id': 'image_set'}, inplace=True)
    
    print('Image set cleaning and analysis completed.')
    for df in all_img_sets_dfs:
        display(df)
    
    output_path = os.path.join(output_folder_path, 'all_image_set_data.xlsx')
    with pd.ExcelWriter(output_path) as writer:
        for i, df in enumerate(all_img_sets_dfs):
            df.to_excel(writer, sheet_name=f'ImageSet_{i+1}', index=False)
        summary_df.to_excel(writer, sheet_name='OverallData', index=False)
        
    return all_img_sets_dfs, summary_df

### Notes on function parameters:
- <code>img_set_paths,</code> Edit the dictionary's paths accordingly. Remember that each image set be a folder containing 3 images: one of the cells, one of the transfection, and one of the p115. NOTE: The pipeline relies on the 'ch00', 'ch01', and 'ch02' at the end of each nuclei image, transfection image, and p115 image, respectively. Ensure that this remains consistently across all image sets.
- <code>output_folder_path,</code> Edit the path accordingly.
- <code>show_viewer=False,</code> IMPORTANT: Only set this to True if you're analyzing a small number of image sets and want to tweak parameters to see how they affect the visual analysis, or if you want to confirm the pipeline's step-by-step output. A Napari window will appear for each image set, so larger sets of like 10+ images will likely affect your computer's performance if you set this parameter to True.
- <code>erosion_intensity=1,</code> Change to 2 or 3 for more intense erosion.
- <code>sigma_est_toggle=False,</code> sigma_est is the estimated noise standard deviation. It multiplies the filter strength to provide a tailored denoising strength. Keep this as False if you don't want it to influence denoising and want more manual control over the denoising strength.
- <code>denoising_strength = 0.8,</code> Remember that this number will by multiplied by sigma_est if sigma_est is set to True, and thus the denoising strength will be 80% of the standard deviation (or 70% if you change this number to 0.7, and so on). Otherwise, tweak this number to see what denoising strength works best.
- <code>denoising_patch_size=11,</code> Larger value = captures more context; better denoising but loses fine detail (which might not matter in some images, but may cause bad nucleus masking in other images darker nucleus pixels).
- <code>denoising_patch_distance=12,</code> Larger value = searches wider area, so likely better denoising but slower processing.
- <code>denoising_fast_mode=True,</code> Make False for more accurate denoising (but it'll be a bit slower).
- <code>nucleus_mask_threshold=0.05,</code> Tweak the threshold to find the optimal value.
- <code>nucleus_area_threshold=500,</code> If there are any isolated regions in the nucleus mask whose areas are less than 500 pixels, they will be deemed not actual nuclei and removed. 500 seems to work well from my own testing on a limited number of images, but tweak this number in case Napari doesn't show smaller nuclei being included in the cleaned nucleus mask.
- <code>desired_donut_width_um=4.5,</code> Change if necessary.
- <code>desired_outer_donut_width_um=1.5,</code> Change if necessary.
- <code>strict_overlapping_filter=False,</code> Setting this to True deletes all cell regions that overlap.
- <code>lenient_overlapping_filter=True,</code> Setting this to True deletes all cell regions whose outer donuts overlap with other regions' inner donuts and vice versa (recommended filter). NOTE: Setting both of these filters to True is redundant and will give priority to the first filter. Do not set both of these filters to False, or the final analysis will likely be inaccurate.
- <code>lower_transfection_intensity_threshold=0.05,</code> Cell regions whose inner donuts have mean transfection intensities that are lower than this number are removed. Tweak the threshold to find the optimal value.
- <code>upper_transfection_intensity_threshold=0.2,</code> Cell regions whose inner donuts have mean transfection intensities that are higher than this number are removed. Tweak the threshold to find the optimal value.
- <code>p115_area_intensity_threshold=0.05</code> Pixels in the p115 image whose intensities are lower than this number are not considered when calculating the total area of p115 (in pixels) for each mask in each cell region. 

In [23]:
img_set_paths = {
    'Image Set 1': './cell-data/image-set-1',
    'Image Set 2': './cell-data/image-set-2'
}

output_folder_path = 'analyzed-cell-data'

In [4]:
clean_and_analyze_images(
    img_set_paths,
    output_folder_path,
    show_viewer=True,
    erosion_intensity=1, 
    sigma_est_toggle=True, 
    denoising_strength = 0.8,
    denoising_patch_size=11,
    denoising_patch_distance=12, 
    denoising_fast_mode=True, 
    nucleus_mask_threshold=0.05, 
    nucleus_area_threshold=500,
    desired_donut_width_um=4.5, 
    desired_outer_donut_width_um=1.5, 
    strict_overlapping_filter=False,
    lenient_overlapping_filter=True, 
    lower_transfection_intensity_threshold=0.05,
    upper_transfection_intensity_threshold=0.2,
    p115_area_intensity_threshold=0.05
)

Analyzing Image Set 1


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Compare these values to the metadata in Fiji ImageJ:
X pixels/µm: 5.5440
Y pixels/µm: 5.5440
Analyzing Image Set 2
Compare these values to the metadata in Fiji ImageJ:
X pixels/µm: 5.5440
Y pixels/µm: 5.5440
Image set cleaning and analysis completed.


,label_id,nucleus_mean_int,nucleus_max_int,nucleus_area,inner_donut_mean_int,inner_donut_max_int,inner_donut_area,outer_donut_mean_int,outer_donut_max_int,outer_donut_area,regional_mean_int,regional_max_int,regional_area
0,29,0.162611,1.000000,48.022521,0.144018,1.000000,33.251366,0.072097,0.247059,3.416236,0.150682,1.000000,84.690124
1,31,0.111972,1.000000,58.824335,0.136644,1.000000,47.046454,0.077057,0.333333,6.149225,0.119688,1.000000,112.020014
2,37,0.111970,0.941176,47.567023,0.107938,0.894118,55.505706,0.079188,0.396078,10.411387,0.106701,0.941176,113.484116
3,48,0.109340,0.894118,59.344905,0.142447,1.000000,65.396523,0.094708,0.764706,12.721413,0.122986,1.000000,137.462841
4,overall_means,0.123973,0.958824,53.439696,0.132762,0.973529,50.300012,0.080762,0.435294,8.174565,0.125014,0.985294,111.914274


,label_id,nucleus_mean_int,nucleus_max_int,nucleus_area,inner_donut_mean_int,inner_donut_max_int,inner_donut_area,outer_donut_mean_int,outer_donut_max_int,outer_donut_area,regional_mean_int,regional_max_int,regional_area
0,7,0.136100,0.980392,43.402469,0.189922,1.0,86.219297,0.150033,1.000000,17.764429,0.168649,1.0,147.386194
1,8,0.121567,0.847059,53.000466,0.232463,1.0,70.797430,0.263864,1.000000,18.154856,0.193886,1.0,141.952751
2,10,0.137843,1.000000,44.768963,0.210093,1.0,80.655712,0.091190,0.545098,13.339589,0.173240,1.0,138.764264
3,30,0.100323,1.000000,51.080866,0.211742,1.0,81.469101,0.126703,1.000000,18.219927,0.160605,1.0,150.769895
4,33,0.155165,1.000000,34.422647,0.201104,1.0,61.589860,0.088565,0.588235,11.289848,0.172589,1.0,107.302355
5,45,0.108606,1.000000,46.005315,0.226637,1.0,71.480677,0.122188,1.000000,13.664945,0.170953,1.0,131.150938
6,overall_means,0.126601,0.971242,45.446788,0.211994,1.0,75.368679,0.140424,0.855556,15.405599,0.173320,1.0,136.221066


([        label_id  nucleus_mean_int  nucleus_max_int  nucleus_area  \
  0             29          0.162611         1.000000     48.022521   
  1             31          0.111972         1.000000     58.824335   
  2             37          0.111970         0.941176     47.567023   
  3             48          0.109340         0.894118     59.344905   
  4  overall_means          0.123973         0.958824     53.439696   
  
     inner_donut_mean_int  inner_donut_max_int  inner_donut_area  \
  0              0.144018             1.000000         33.251366   
  1              0.136644             1.000000         47.046454   
  2              0.107938             0.894118         55.505706   
  3              0.142447             1.000000         65.396523   
  4              0.132762             0.973529         50.300012   
  
     outer_donut_mean_int  outer_donut_max_int  outer_donut_area  \
  0              0.072097             0.247059          3.416236   
  1              0.07705

## **Initial version** of cell image filtering and p115 analysis

In [5]:
import napari
import numpy as np
from skimage import morphology, restoration, filters, measure
from tifffile import imread

#img = imread('cell-data/parent_control_z3_ch00.tif') --> This was an initial testing image

# Current testing images
#nuclei_img = imread('cell-data/test-image-set/22KO_mCHC17_GFP_3_z3_ch00.tif')
#transfection_img = imread('cell-data/test-image-set/22KO_mCHC17_GFP_3_z3_ch01.tif')
#p115_img = imread('cell-data/test-image-set/22KO_mCHC17_GFP_3_z3_ch02.tif')

nuclei_img = imread('cell-data/image-set-2/parent_mCHC17_GFP_4_z3_ch00.tif')
transfection_img = imread('cell-data/image-set-2/parent_mCHC17_GFP_4_z2_ch01.tif')
p115_img = imread('cell-data/image-set-2/parent_mCHC17_GFP_4_z2_ch02.tif')

viewer = napari.Viewer()
viewer.add_image(nuclei_img, name='nuclei')
viewer.theme = 'light'

### Erosion

In [67]:
from skimage.morphology import erosion, disk

eroded = erosion(nuclei_img, disk(1)) # Change to 2 or 3 for more intense erosion

viewer.add_image(eroded, name='eroded')

<Image layer 'eroded' at 0x315f25520>

### Estimate noise and denoise via non-local means

In [68]:
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.util import img_as_float

img_float = img_as_float(eroded)

sigma_est = np.mean(estimate_sigma(img_float, channel_axis=None)) # Estimated noise standard deviation. Lower SD means lower estimated noise.

denoised = denoise_nl_means(
    img_float,
    h=0.8 * sigma_est, # h is filter strength. This line sets filter strength to 80% of estimated noise level, but tweak this number if necessary.
    patch_size=11, # Larger value = captures more context; better denoising but loses fine detail (probably doesn't matter for this use).
    patch_distance=12, # Larger value = searches wider area, so better denoising but slower processing.
    fast_mode=True # Make False for more accurate denoising (but it'll be a bit slower).
)

viewer.add_image(denoised, name='denoised')

<Image layer 'denoised' at 0x394cdf680>

### Create nucleus mask

In [69]:
threshold = 0.05 # Play around with the threshold to find the optimal value.
initial_nuclei_mask = denoised > threshold # For each pixel --> if it's brighter than the threshold value, count it as foreground (part of a nucleus).
# The line above creates a boolean array.

viewer.add_labels(initial_nuclei_mask.astype(int), name='initial_nuclei_mask') # This turns the boolean array into binaries.

<Labels layer 'initial_nuclei_mask' at 0x3ccb39c40>

### Fill any gaps in the mask regions

In [70]:
from scipy.ndimage import binary_fill_holes
import numpy as np

# Assuming 'mask' is a binary numpy array where nuclei are True (1) and background is False (0)
filled_mask = binary_fill_holes(initial_nuclei_mask)

# Convert back to int if needed
filled_mask = filled_mask.astype(np.uint8)

viewer.add_labels(filled_mask, name='hole-filled-mask')

<Labels layer 'hole-filled-mask' at 0x32f46f710>

### Separate mask into distinct regions with integer labels

In [71]:
from skimage.measure import label

labeled_mask = label(filled_mask) # Label connected regions of the mask

viewer.add_labels(labeled_mask, name='labeled_nuclei_mask')

<Labels layer 'labeled_nuclei_mask' at 0x3948e74d0>

### Clean mask by removing nucleus regions that are cut off by the image and by removing isolated pixels that remain after erosion and denoising

In [72]:
from skimage.measure import regionprops
import numpy as np

image_height, image_width = labeled_mask.shape
cleaned_nuclei_mask = np.zeros_like(labeled_mask)

for region in regionprops(labeled_mask):
    min_row, min_column, max_row, max_column = region.bbox

    # This checks for labeled regions at the edges of the image
    touches_edge = (min_row == 0 or min_column == 0 or max_row == image_height or max_column == image_width)
    
    if not touches_edge and region.area >= 500:
        cleaned_nuclei_mask[labeled_mask == region.label] = region.label 
        # This line sets all pixels in cleaned_nuclei_mask at positions where the boolean mask is True to the integer value region.label.
        # Each specific integer value is a specific label/color.
        
viewer.add_labels(cleaned_nuclei_mask, name='cleaned_nuclei_mask')

<Labels layer 'cleaned_nuclei_mask' at 0x32bf6c3b0>

### Determine the image's microns per pixel ratio and create the first donut mask --> PERSONAL NOTE: Confirm the output with Fiji since this is a new image

In [73]:
from skimage.morphology import dilation, disk
from skimage.segmentation import find_boundaries
import tifffile

with tifffile.TiffFile('cell-data/image-set-2/22KO_mCHC17_GFP_3_z3_ch00.tif') as tif: # CHANGE THIS DEPENDING ON THE IMAGE SET YOU'RE USING!!!
    page = tif.pages[0]
    tags = page.tags

    x_res = tags['XResolution'].value  # (num, denom)
    y_res = tags['YResolution'].value
    res_unit = tags['ResolutionUnit'].value  # 3 = centimeters

    # Convert to pixels per micrometer
    if res_unit == 3:  # Centimeters
        x_ppcm = x_res[0] / x_res[1]
        y_ppcm = y_res[0] / y_res[1]
        x_ppum = x_ppcm / 10000  # cm → µm
        y_ppum = y_ppcm / 10000
        print('Compare these values to the metadata in Fiji ImageJ:')
        print(f'X pixels/µm: {x_ppum:.4f}')
        print(f'Y pixels/µm: {y_ppum:.4f}')
    else:
        print('Unsupported resolution unit:', res_unit)

microns_per_pixel = (1 / x_ppum + 1 / y_ppum) / 2
desired_donut_width_um = 4.5
n_pixels = int(desired_donut_width_um / microns_per_pixel)

outer_mask = dilation(cleaned_nuclei_mask, disk(n_pixels))

donut_mask = outer_mask - cleaned_nuclei_mask
donut_mask[donut_mask < 0] = 0  # Just in case

donut_mask = donut_mask.astype(np.uint8)

viewer.add_labels(donut_mask, name='donut')

# For calculating the area later on:
microns_per_pixel_x = 1 / x_ppum
microns_per_pixel_y = 1 / y_ppum

Compare these values to the metadata in Fiji ImageJ:
X pixels/µm: 5.5440
Y pixels/µm: 5.5440


### Create the outer donut mask

In [74]:
desired_outer_donut_width_um = 1.5
n_pixels = int(desired_outer_donut_width_um / microns_per_pixel)

outer_donut_mask = np.zeros_like(cleaned_nuclei_mask, dtype=np.uint16)

regions = regionprops(cleaned_nuclei_mask)

for region in regions:
    nucleus_mask = (cleaned_nuclei_mask == region.label)
    first_donut_mask = (donut_mask == region.label)
    excluded_area = nucleus_mask | first_donut_mask
    outer_mask = dilation(excluded_area, disk(n_pixels))
    outer_donut = outer_mask & (~excluded_area)
    outer_donut_mask[outer_donut] = region.label

viewer.add_labels(outer_donut_mask, name='outer_donut')

<Labels layer 'outer_donut' at 0x3ccda4770>

### **Initial overlapping method (more extreme)**: Delete all overlapping regions 

In [10]:
from skimage.morphology import square

# Step 1: Get all unique labels
labels = np.unique(cleaned_nuclei_mask)
labels = labels[labels != 0]

# Step 2: Build combined masks per region
regions = []
for label_id in labels:
    combined_mask = (
        (cleaned_nuclei_mask == label_id) |
        (donut_mask == label_id) |
        (outer_donut_mask == label_id)
    )
    regions.append({
        'label': label_id,
        'mask': combined_mask
    })

# Step 3: Find touching regions using dilation
labels_to_remove = set()

for i, region_a in enumerate(regions):
    dilated_a = dilation(region_a['mask'], square(3))  # 8-connected dilation

    for j, region_b in enumerate(regions):
        if i >= j:
            continue  # avoid duplicate checks

        if np.any(dilated_a & region_b['mask']):
            labels_to_remove.add(region_a['label'])
            labels_to_remove.add(region_b['label'])

# Step 4: Remove touching labels from all masks
def remove_labels(mask, labels_to_remove):
    output = np.copy(mask)
    output[np.isin(mask, list(labels_to_remove))] = 0
    return output.astype(np.int32)

cleaned_filtered = remove_labels(cleaned_nuclei_mask, labels_to_remove)
donut_filtered = remove_labels(donut_mask, labels_to_remove)
outer_donut_filtered = remove_labels(outer_donut_mask, labels_to_remove)

viewer.add_labels(cleaned_filtered, name='no_overlap_nuclei')
viewer.add_labels(donut_filtered, name='no_overlap_donuts')
viewer.add_labels(outer_donut_filtered, name='no_overlap_outer_donut')

/var/folders/zc/r5_5f1x56sq9gtmyssh_0g300000gn/T/ipykernel_2156/2779773151.py:24: FutureWarning: `square` is deprecated since version 0.25 and will be removed in version 0.27. Use `skimage.morphology.footprint_rectangle` instead.
  dilated_a = dilation(region_a['mask'], square(3))  # 8-connected dilation


<Labels layer 'no_overlap_outer_donut' at 0x32ab75340>

### **Alternative overlapping method (more lenient)**: Remove regions whose outer donuts overlap with other regions' inner donuts and vice versa

In [75]:
from skimage.morphology import binary_dilation, square

labels = np.unique(cleaned_nuclei_mask)
labels = labels[labels != 0]

regions = []
for label_id in labels:
    region = {
        'label': label_id,
        'nucleus': (cleaned_nuclei_mask == label_id),
        'donut': (donut_mask == label_id),
        'outer': (outer_donut_mask == label_id)
    }
    regions.append(region)

labels_to_remove = set()

for i, region_a in enumerate(regions):
    inner_donut_a = region_a['donut']
    outer_donut_a = region_a['outer']

    for j, region_b in enumerate(regions):
        if i == j:
            continue

        inner_donut_b = region_b['donut']
        outer_donut_b = region_b['outer']

        # Check for inner donut of A overlapping outer donut of B
        if np.any(inner_donut_a & outer_donut_b):
            labels_to_remove.add(region_a['label'])
            labels_to_remove.add(region_b['label'])

        # Check for outer donut of A overlapping inner donut of B
        elif np.any(outer_donut_a & inner_donut_b):
            labels_to_remove.add(region_a['label'])
            labels_to_remove.add(region_b['label'])

# Remove only labels that meet this specific donut overlap condition
def remove_labels(mask, labels_to_remove):
    output = np.copy(mask)
    output[np.isin(mask, list(labels_to_remove))] = 0
    return output.astype(np.int32)

cleaned_filtered = remove_labels(cleaned_nuclei_mask, labels_to_remove)
donut_filtered = remove_labels(donut_mask, labels_to_remove)
outer_donut_filtered = remove_labels(outer_donut_mask, labels_to_remove)

viewer.add_labels(cleaned_filtered, name='filtered_nuclei')
viewer.add_labels(donut_filtered, name='filtered_donuts')
viewer.add_labels(outer_donut_filtered, name='filtered_outer_donuts')

<Labels layer 'filtered_outer_donuts' at 0x3cfb37410>

### Delete regions with cut-off donuts

In [76]:
image_height, image_width = cleaned_filtered.shape
pre_tf_nuclei = np.zeros_like(cleaned_filtered)
pre_tf_donuts = np.zeros_like(donut_filtered)
pre_tf_outer_donuts = np.zeros_like(outer_donut_filtered)

for region in regions: # Using the list from the previous cell
    label_id = region['label']
    
    combined = (
        (cleaned_filtered == label_id) |
        (donut_filtered == label_id) |
        (outer_donut_filtered == label_id)
    )

    coords = np.argwhere(combined)
    if coords.size == 0:
        continue
    min_row, min_col = coords.min(axis=0)
    max_row, max_col = coords.max(axis=0)

    touches_edge = (min_row == 0 or min_col == 0 or max_row == image_height or max_col == image_width)

    if not touches_edge:
        pre_tf_nuclei[cleaned_filtered == label_id] = label_id
        pre_tf_donuts[donut_filtered == label_id] = label_id
        pre_tf_outer_donuts[outer_donut_filtered == label_id] = label_id

viewer.add_labels(pre_tf_nuclei, name='pre_transfection_nuclei')
viewer.add_labels(pre_tf_donuts, name='pre_transfection_donuts')
viewer.add_labels(pre_tf_outer_donuts, name='pre_transfection_outer_donuts')

<Labels layer 'pre_transfection_outer_donuts' at 0x37b8e9910>

In [77]:
# If these pairs of numbers are different, the cell above didn't work properly.
print(image_height, image_width, '|', combined.shape)

1024 1024 | (1024, 1024)


### Remove cells with low transfection (by evaluating mean intensity in the inner donuts against upper and lower intensity thresholds)

In [78]:
# This is purely for visualization, so no need to mess with these parameters too much.
viewer.add_image(
    transfection_img,
    name='transfection',
    colormap='green',         
    contrast_limits=[0, 255], # adjust depending on intensity range
    blending='additive',     
    opacity=0.7 # adjust as needed
)

<Image layer 'transfection' at 0x3663c1730>

In [79]:
transfection_img_norm = (transfection_img - transfection_img.min()) / (transfection_img.max() - transfection_img.min())

# Raise or lower these as needed:
lower_threshold = 0.05  
upper_threshold = 0.2

labels = np.unique(pre_tf_nuclei)
labels = labels[labels != 0]

final_nuclei = np.zeros_like(pre_tf_nuclei)
final_donuts = np.zeros_like(pre_tf_donuts)
final_outer_donuts = np.zeros_like(pre_tf_outer_donuts)

for label_id in labels:
    nucleus_mask = (pre_tf_nuclei == label_id)
    donut_mask = (pre_tf_donuts == label_id)
    outer_mask = (pre_tf_outer_donuts == label_id)

    mean_intensity = transfection_img_norm[donut_mask].mean()

    if mean_intensity >= lower_threshold and mean_intensity <= upper_threshold:
        final_nuclei[nucleus_mask] = label_id
        final_donuts[donut_mask] = label_id
        final_outer_donuts[outer_mask] = label_id

viewer.add_labels(final_nuclei, name='final_nuclei')
viewer.add_labels(final_donuts, name='final_donuts')
viewer.add_labels(final_outer_donuts, name='final_outer_donuts')

<Labels layer 'final_outer_donuts' at 0x3fc1cb9e0>

### Evaluate p115 intensity

p115 additions:
- mean intensity, max intensity (brightest pixel), and area for each mask and big mask

Within regions_stats:
Each key is a region. Each value is a list holding the stats of that region, with information in the following order:
- [integer label,
- a ColorArray with the RGBA values that correspond to that integer label,
- {nucleus mask: [mean intensity, max intensity, p115 area in mask]}
- {inner donut mask: [mean intensity, max intensity, p115 area in mask]}
- {outer donut mask: [mean intensity, max intensity, p115 area in mask]}
- {total/regional mask: mean [intensity, max intensity, p115 area in mask]}]

In [80]:
viewer.add_image(
    p115_img,
    name='p115',
    colormap='magma',         
    contrast_limits=[0, 255], # adjust depending on intensity range
    blending='additive',     
    opacity=0.7 # adjust as needed
)

<Image layer 'p115' at 0x40994f6e0>

In [86]:
threshold = 0.05
p115_img_norm = (p115_img - p115_img.min()) / (p115_img.max() - p115_img.min())
regions_stats = {}

layer = viewer.layers['final_nuclei']
labels = np.unique(layer.data)
labels = labels[labels != 0]
label_colors = {label: layer.get_color(label) for label in labels}

for label, color in label_colors.items():
    print(f'Label {label}: {color}')

for label_id in labels:
    nucleus_mask = (final_nuclei == label_id)
    donut_mask = (final_donuts == label_id)
    outer_donut_mask = (final_outer_donuts == label_id)
    regional_mask = nucleus_mask | donut_mask | outer_donut_mask
    
    region_info = {
        'label_color': label_colors[label_id],
        'nucleus_mask_stats': [],
        'donut_mask_stats': [],
        'outer_donut_mask_stats': [],
        'regional_mask_stats': [],
    }

    for mask, mask_name in zip([nucleus_mask, donut_mask, outer_donut_mask, regional_mask], 
                               ['nucleus_mask', 'donut_mask', 'outer_donut_mask', 'regional_mask']):
        key = f'{mask_name}_stats'
        p115_overlap_pixels = p115_img_norm[(mask) & (p115_img_norm > 0)]
        region_info[key].append(p115_overlap_pixels.mean() if p115_overlap_pixels.size > 0 else 0)
        region_info[key].append(p115_overlap_pixels.max() if p115_overlap_pixels.size > 0 else 0)
        region_info[key].append((p115_overlap_pixels > threshold).sum() * microns_per_pixel_x * microns_per_pixel_y)
    
    regions_stats[label_id] = region_info

Label 7: [0.56078434 0.6784314  0.69803923 1.        ]
Label 8: [0.5254902 0.5647059 0.6039216 1.       ]
Label 10: [0.96862745 0.26666668 0.23137255 1.        ]
Label 30: [0.3764706  0.47843137 0.20392157 1.        ]
Label 33: [0.61960787 0.5294118  0.43529412 1.        ]
Label 45: [0.6862745  0.29411766 0.9490196  1.        ]


In [104]:
import pandas as pd

regions_df = pd.DataFrame(regions_stats).T.reset_index().rename(columns={'index': 'label_id'})
regions_df

,label_id,label_color,nucleus_mask_stats,donut_mask_stats,outer_donut_mask_stats,regional_mask_stats
0,7,"[0.56078434, 0.6784314, 0.69803923, 1.0]","[0.13609955069673063, 0.9803921568627451, 43.4...","[0.18992244296562555, 1.0, 86.21929660055089]","[0.1500326797385621, 1.0, 17.764428658075772]","[0.16864900821330572, 1.0, 147.38619381150775]"
1,8,"[0.5254902, 0.5647059, 0.6039216, 1.0]","[0.12156699618283254, 0.8470588235294118, 53.0...","[0.23246287961094725, 1.0, 70.79742996332028]","[0.2638641680534452, 1.0, 18.154855661549963]","[0.19388639437211913, 1.0, 141.9527513464919]"
2,10,"[0.96862745, 0.26666668, 0.23137255, 1.0]","[0.13784333274685745, 1.0, 44.768963065040765]","[0.21009297071856806, 1.0, 80.65571180104365]","[0.09118994162550514, 0.5450980392156862, 13.3...","[0.17323980204056483, 1.0, 138.76426415145266]"
3,30,"[0.3764706, 0.47843137, 0.20392157, 1.0]","[0.10032325806375499, 1.0, 51.08086628787355]","[0.21174235439402322, 1.0, 81.4691013916149]","[0.126703146374829, 1.0, 18.21992682879566]","[0.160605437161749, 1.0, 150.7698945082841]"
4,33,"[0.61960787, 0.5294118, 0.43529412, 1.0]","[0.15516516951608228, 1.0, 34.42264747297466]","[0.2011040193203381, 1.0, 61.58985979805391]","[0.08856486567462299, 0.5882352941176471, 11.2...","[0.17258949724394917, 1.0, 107.3023547881573]"
5,45,"[0.6862745, 0.29411766, 0.9490196, 1.0]","[0.10860600905522624, 1.0, 46.005315242709045]","[0.2266368760994902, 1.0, 71.48067721940012]","[0.12218844984802431, 1.0, 13.664945121596746]","[0.17095259369570612, 1.0, 131.1509375837059]"


In [105]:
stat_cols = [
    'nucleus_mask_stats',
    'donut_mask_stats',
    'outer_donut_mask_stats',
    'regional_mask_stats'
]

summary_row = {
    'label_id': 'overall',
    'label_color': None,
}

for col in stat_cols:
    stats_array = np.stack(regions_df[col].values)
    mean_stats = stats_array.mean(axis=0)
    summary_row[col] = mean_stats.tolist()

regions_df = pd.concat([regions_df, pd.DataFrame([summary_row])], ignore_index=True)
regions_df

,label_id,label_color,nucleus_mask_stats,donut_mask_stats,outer_donut_mask_stats,regional_mask_stats
0,7,"[0.56078434, 0.6784314, 0.69803923, 1.0]","[0.13609955069673063, 0.9803921568627451, 43.4...","[0.18992244296562555, 1.0, 86.21929660055089]","[0.1500326797385621, 1.0, 17.764428658075772]","[0.16864900821330572, 1.0, 147.38619381150775]"
1,8,"[0.5254902, 0.5647059, 0.6039216, 1.0]","[0.12156699618283254, 0.8470588235294118, 53.0...","[0.23246287961094725, 1.0, 70.79742996332028]","[0.2638641680534452, 1.0, 18.154855661549963]","[0.19388639437211913, 1.0, 141.9527513464919]"
2,10,"[0.96862745, 0.26666668, 0.23137255, 1.0]","[0.13784333274685745, 1.0, 44.768963065040765]","[0.21009297071856806, 1.0, 80.65571180104365]","[0.09118994162550514, 0.5450980392156862, 13.3...","[0.17323980204056483, 1.0, 138.76426415145266]"
3,30,"[0.3764706, 0.47843137, 0.20392157, 1.0]","[0.10032325806375499, 1.0, 51.08086628787355]","[0.21174235439402322, 1.0, 81.4691013916149]","[0.126703146374829, 1.0, 18.21992682879566]","[0.160605437161749, 1.0, 150.7698945082841]"
4,33,"[0.61960787, 0.5294118, 0.43529412, 1.0]","[0.15516516951608228, 1.0, 34.42264747297466]","[0.2011040193203381, 1.0, 61.58985979805391]","[0.08856486567462299, 0.5882352941176471, 11.2...","[0.17258949724394917, 1.0, 107.3023547881573]"
5,45,"[0.6862745, 0.29411766, 0.9490196, 1.0]","[0.10860600905522624, 1.0, 46.005315242709045]","[0.2266368760994902, 1.0, 71.48067721940012]","[0.12218844984802431, 1.0, 13.664945121596746]","[0.17095259369570612, 1.0, 131.1509375837059]"
6,overall,None,"[0.126600719376914, 0.9712418300653595, 45.446...","[0.21199359051816538, 1.0, 75.36867946233063]","[0.14042387521916477, 0.8555555555555555, 15.4...","[0.17332045545456565, 1.0, 136.22106603159992]"


In [1]:
import os
import glob
import re

img_set_paths = {
    'Image Set 1': './cell-data/image-set-1',
    'Image Set 2': './cell-data/image-set-2'
}

for img_set, img_set_path in img_set_paths.items():
    print(f'Analyzing {img_set}')
    tif_files = glob.glob(os.path.join(img_set_path, '*.tif'))
    sorted_tif_files = sorted(tif_files, key=lambda f: re.search(r'ch\d+', f).group())
    print(sorted_tif_files)

output_folder_path = 'analyzed-cell-data'

Analyzing Image Set 1
['./cell-data/image-set-1/22KO_mCHC17_GFP_3_z3_ch00.tif', './cell-data/image-set-1/22KO_mCHC17_GFP_3_z3_ch01.tif', './cell-data/image-set-1/22KO_mCHC17_GFP_3_z3_ch02.tif']
Analyzing Image Set 2
['./cell-data/image-set-2/parent_mCHC17_GFP_4_z3_ch00.tif', './cell-data/image-set-2/parent_mCHC17_GFP_4_z2_ch01.tif', './cell-data/image-set-2/parent_mCHC17_GFP_4_z2_ch02.tif']
